In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.63723018613394, 0.7007651709662439)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_six_month'].values

## columns 
cols = KY_X.columns

In [9]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.1, cols, 816)
single_stump_model['features']

['age_at_current_charge19.0',
 'age_at_current_charge22.0',
 'age_at_current_charge29.0',
 'age_at_current_charge36.0',
 'age_at_current_charge48.0',
 'age_at_current_charge62.0',
 'age_at_current_charge69.0',
 'age_at_current_charge70.0',
 'p_charges1',
 'p_charges2',
 'p_charges6',
 'p_violence2',
 'p_misdemeanor3',
 'p_property2',
 'p_weapon1',
 'p_drug2',
 'p_dui1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'three_year1']

In [10]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [11]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.1}, 0.6777214980063466, 0.0437942067580076)

### RiskSLIM

In [12]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['general_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['general_six_month'].values

## columns 
cols = KY_X.columns

In [14]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 
                                        0.05, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

13

In [15]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [16]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 04:01 PM | 183 rows in lookup table
11/12/19 @ 04:01 PM | ------------------------------------------------------------
11/12/19 @ 04:01 PM | runnning initialization procedure
11/12/19 @ 04:01 PM | ------------------------------------------------------------
11/12/19 @ 04:01 PM | CPA produced 2 cuts
11/12/19 @ 04:01 PM | running naive rounding on 4 solutions
11/12/19 @ 04:01 PM | best objective value: 0.6931
11/12/19 @ 04:01 PM | rounding produced 4 integer solutions
11/12/19 @ 04:01 PM | best objective value is 0.6931
11/12/19 @ 04:01 PM | running sequential rounding on 4 solutions
11/12/19 @ 04:01 PM | best objective value: 0.6931
11/12/19 @ 04:01 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 04:01 PM | polishing 4 solutions
11/12/19 @ 04:01 PM | best objective value: 0.6931
11/12/19 @ 04:01 PM | polishing produced 4 integer solutions
11/12/19 @ 04:01 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:01 PM | adding 157 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5314.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 1882722 93763        cutoff              0.5159        0.5087  5736034    1.40%           rho_6 U 1882722 1882721     51
 1940157 96646        cutoff              0.5159        0.5087  5821173    1.40%           rho_7 U 1940157 1940156     51
 1996446 95488        0.5122     1        0.5159        0.5087  5919019    1.39%           rho_0 U 1996446 1996445     53
 2048606 90995        cutoff              0.5159        0.5091  6028191    1.32%           rho_0 U 2048606 2048605     51
 2104161 90382        0.5100     1        0.5159        0.5092  6115343    1.30%           rho_7 D 2104161 2104159     47
 2154605 92608        0.5137     4        0.5159        0.5093  6210473    1.27%           rho_9 U 2154605 2154604     43
 2204632 87816        cutoff              0.5159        0.5096  6304902    1.22%           rho_0 D 2204632 257423     50
Elapsed time = 216.30 sec. (168093.19 ticks, tree = 22.09 MB, solutions = 6)
 2259263 90133        0.5096     1        0.5159        0.5096  641535

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:06 PM | adding 165 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5098.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 1383592 240748        0.4990     3        0.5075        0.4909  6737754    3.26%          rho_10 U 1383592 1383591     40
 1424690 249217        cutoff              0.5075        0.4910  6846253    3.24%           rho_9 U 1424690 1424688     42
 1464230 257711        0.5043     9        0.5075        0.4912  6982994    3.20%           rho_3 N 1464230 1464229     33
 1502274 261737        0.4915     9        0.5075        0.4913  7114668    3.17%           rho_7 D 1502274 1502272     36
 1542284 265601        0.4930     8        0.5075        0.4915  7244372    3.15%           rho_4 U 1542284 1542282     56
 1581156 269610        0.4918     6        0.5075        0.4918  7366856    3.09%           rho_3 U 1581156 1550590     38
*1589088 269825      integral     0        0.5074        0.4918  7390856    3.08%
 1624185 280966        0.4918     8        0.5074        0.4918  7498671    3.07%           rho_7 D 1624185 1566357     31
Elapsed time = 209.17 sec. (168095.09 ticks, tree = 70.25

...

 1264896 159281        0.5222     2        0.5240        0.5053  5852345    3.57%           rho_0 U 1264896 1264894     33
 1294680 162434        cutoff              0.5240        0.5053  6006392    3.57%           rho_8 U 1294680 1294678     37
 1326978 165572        0.5053     6        0.5240        0.5053  6165066    3.57%           rho_0 D 1326978 1326976     31
 1357386 168535        0.5122     1        0.5240        0.5053  6308644    3.57%           rho_7 U 1357386 1342341     17
 1388702 171796        0.5204     8        0.5240        0.5053  6432576    3.57%           rho_3 N 1388702 1388701     31
 1418314 175721        0.5053     9        0.5240        0.5053  6574603    3.57%           rho_3 U 1418314 1418312     38
 1447694 178733        0.5053     9        0.5240        0.5053  6721724    3.57%           rho_3 U 1447694 1447692     41
Elapsed time = 205.98 sec. (168095.59 ticks, tree = 44.61 MB, solutions = 7)
 1476568 181635        0.5053     7        0.5240        0.505

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:26 PM | adding 154 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5326.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 1305869 185837        cutoff              0.5194        0.5015  6694207    3.45%           rho_1 U 1305869 1305867     34
 1337938 185910        0.5015     8        0.5194        0.5015  6847416    3.45%           rho_7 D 1337938 1337936     40
 1368318 188185        0.5024     9        0.5194        0.5015  6996585    3.45%           rho_3 D 1368318 1368317     31
 1398777 191811        cutoff              0.5194        0.5015  7158542    3.45%           rho_9 U 1398777 1398776     36
 1430902 196725        0.5067     7        0.5194        0.5015  7335794    3.45%           rho_5 N 1430902 1430901     34
 1462438 199557        0.5053     8        0.5194        0.5015  7499815    3.45%           rho_7 U 1462438 1462437     42
 1493230 200725        0.5174     7        0.5194        0.5015  7654858    3.45%           rho_1 U 1493230 1493228     53
 1526940 204854        cutoff              0.5194        0.5015  7825237    3.45%          rho_11 D 1526940 1526939     41
 1558847 209251 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 04:36 PM | adding 155 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5308.
Tried aggregator 1 time.
Reduced MIP has 28 rows, 30 columns, and 81 nonzeros.
Reduced MIP has 13 binaries, 15 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 1276522 214607        0.5017     1        0.5183        0.5014  5751155    3.27%           rho_0 D 1276522 1276520     27
 1313425 216484        cutoff              0.5183        0.5016  5856683    3.23%           rho_3 D 1313425 1313424     49
 1351049 218042        0.5083     1        0.5183        0.5018  5954407    3.18%           rho_7 D 1351049 1351037     28
 1388816 220749        cutoff              0.5183        0.5020  6043423    3.15%          rho_11 D 1388816 1388814     25
 1423842 220834        0.5160     5        0.5183        0.5025  6148109    3.06%           rho_2 D 1423842 1423841     37
 1460151 224186        0.5077     6        0.5183        0.5025  6247785    3.05%           rho_3 U 1460151 1460149     33
 1492548 223228        0.5108     8        0.5183        0.5027  6356517    3.01%           rho_7 U 1492548 1492547     33
Elapsed time = 216.08 sec. (168096.15 ticks, tree = 54.66 MB, solutions = 7)
 1527901 226519        0.5060     1        0.5183        0.502

In [18]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.6210030908224663, 0.5674001925595513)

In [19]:
#### save results
summary_general6_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}

In [21]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM"],
           ["General",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)